<a href="https://colab.research.google.com/github/kobemawu/www/blob/master/LDA_TA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nltkの文書群にgensimのLDAを適用してみる

## 準備

必要ライブラリ(全部pipで入ります)
 * nltk
 * gensim
 * pyLDAvis

In [0]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
#nltk使ったことない人は、pip install してから対話環境等で以下のdataset等をダウンロードしてください
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("reuters")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## データロード・前処理

In [0]:
#dataset読み込み
from nltk.corpus import reuters as corpus

### 今回はこういう文書(をBOW化したもの)を用います

In [0]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears  
among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They  
told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And  
lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long -  
run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will  
impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to  
stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at  
10 billion dlrs and spokesmen for major electronics firms said they would virtually halt exports 

In [0]:
#全document数
len(corpus.fileids())

10788

In [0]:

#前からk個のdocumentのみで学習する場合
#k=1000
#docs=[corpus.words(fileid) for fileid in corpus.fileids()[:k]]

#全documentで学習する場合
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...], ['CHINA', 'DAILY', 'SAYS', 'VERMIN', 'EAT', '7', '-', ...], ['JAPAN', 'TO', 'REVISE', 'LONG', '-', 'TERM', ...], ['THAI', 'TRADE', 'DEFICIT', 'WIDENS', 'IN', 'FIRST', ...], ['INDONESIA', 'SEES', 'CPO', 'PRICE', 'RISING', ...]]
num of docs: 10788


## 前処理 

In [0]:
#ストップワードリストの作成

#1 nltkのストップワードリスト
en_stop = nltk.corpus.stopwords.words('english')

# 一度LDAしてみる等して，適宜ノイズになってそうな記号等を見つけて，ストップワードリストに新たに加える
#【発展】記号や数字は正規表現で消してみましょう
en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?",">","<"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price"]          \
         +en_stop

In [0]:
#前処理関数の作成

from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [0]:
#before
print(docs[0][:25]) 

#after
print(preprocess_documents(docs)[0][:25])

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears']
['asian', 'exporter', 'fear', 'damage', 'japan', 'rift', 'mounting', 'trade', 'friction', 'japan', 'raise', 'fear', 'among', 'many', 'asia', 'exporting', 'nation', 'row', 'could', 'inflict', 'far', 'reaching', 'economic', 'damage', 'businessmen']


## LDA準備

In [0]:
import gensim
from gensim import corpora

In [0]:
#documentを，gensim LDAが読み込めるデータ構造にする

#辞書の作成
dictionary = corpora.Dictionary(preprocess_documents(docs))
#コーパスの作成
corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [0]:
#Dictionary:gensimにおける辞書クラス
#token2id属性には単語と辞書IDとの対応が格納される

print(dictionary.token2id)

{'15': 0, '17': 1, '1985': 2, '30': 3, '300': 4, '53': 5, '95': 6, '>.': 7, 'able': 8, 'accounting': 9, 'action': 10, 'advantage': 11, 'allege': 12, 'allow': 13, 'also': 14, 'american': 15, 'among': 16, 'analyst': 17, 'april': 18, 'asia': 19, 'asian': 20, 'ask': 21, 'association': 22, 'australia': 23, 'australian': 24, 'avow': 25, 'await': 26, 'aware': 27, 'barrier': 28, 'beef': 29, 'beyond': 30, 'big': 31, 'block': 32, 'boost': 33, 'broker': 34, 'budget': 35, 'business': 36, 'businessmen': 37, 'button': 38, 'call': 39, 'canberra': 40, 'capel': 41, 'capital': 42, 'centre': 43, 'chairman': 44, 'chief': 45, 'co': 46, 'coal': 47, 'commercial': 48, 'complete': 49, 'concern': 50, 'conflict': 51, 'continue': 52, 'correspondent': 53, 'cost': 54, 'could': 55, 'country': 56, 'curb': 57, 'cut': 58, 'damage': 59, 'day': 60, 'defuse': 61, 'democratic': 62, 'deputy': 63, 'despite': 64, 'deterioration': 65, 'diplomatic': 66, 'director': 67, 'disadvantage': 68, 'dispute': 69, 'domestically': 70, 'due

In [0]:
#corpusにはdocumentごとに単語の(ID、出現回数)のリストが得られる

print(corpus_[0][:10]) #文章での出現順でなく辞書IDの若い順なことに注意

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1)]


In [0]:
#before
print([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]])

#after
print(dictionary.doc2bow([w.lower() for w in corpus.sents(corpus.fileids()[0])[0]]))

#これを全文書の全文に適用したのがcorpus_

['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[(16, 1), (19, 1), (20, 1), (37, 1), (55, 1), (59, 2), (72, 1), (85, 1), (88, 1), (89, 1), (96, 1), (116, 1), (120, 2), (142, 1), (157, 1), (198, 1), (209, 1), (210, 1), (256, 1)]


## LDA学習

In [0]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

## パラメータの確認

In [0]:
#(トピックID, 当該トピックにおける単語とそのprobability)  ※　のうち、上位num_words位 

topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.012*"corp" + 0.011*"ltd" + 0.010*"china" + 0.010*"unit" + 0.009*"steel" + 0.009*"inc" + 0.009*"co" + 0.008*"sell" + 0.008*"plant" + 0.008*"venture" + 0.008*"product" + 0.007*"south" + 0.006*"subsidiary" + 0.006*"buy" + 0.006*"group"')
(1, '0.014*"would" + 0.013*"trade" + 0.008*"export" + 0.007*"state" + 0.006*"ec" + 0.006*"import" + 0.006*"official" + 0.005*"last" + 0.005*"bill" + 0.005*"producer" + 0.005*"tax" + 0.005*"program" + 0.005*"country" + 0.005*"government" + 0.004*"tell"')
(2, '0.036*"record" + 0.034*"dividend" + 0.034*"april" + 0.022*"pay" + 0.020*"div" + 0.020*"qtly" + 0.019*"split" + 0.018*"march" + 0.018*"prior" + 0.017*"set" + 0.017*"may" + 0.016*"stock" + 0.015*"quarterly" + 0.014*"payable" + 0.012*"15"')
(3, '0.013*"crown" + 0.006*"bank" + 0.006*"norway" + 0.005*"bill" + 0.005*"utility" + 0.005*"cost" + 0.004*"nuclear" + 0.004*"mature" + 0.004*"gas" + 0.004*"note" + 0.004*"50" + 0.004*"development" + 0.004*"icn" + 0.004*"png" + 0.004*"group"')
(4, '0.018*"offer

In [0]:
#[(当該documentにおけるトピックIDとそのprobability　)]　 ※　のうち、minimum_probabilityの値を超えるもの

for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(0, 0.115813754), (1, 0.19862457), (5, 0.042820185), (10, 0.015951043), (18, 0.6231575)]
document ID 1:[(0, 0.10127618), (7, 0.2583429), (19, 0.6105497)]
document ID 2:[(3, 0.24605535), (5, 0.043198828), (7, 0.032727987), (12, 0.04823864), (18, 0.37129283), (19, 0.2459836)]
document ID 3:[(1, 0.011728283), (8, 0.37921113), (10, 0.17647164), (13, 0.020077957), (18, 0.26147315), (19, 0.13614109)]
document ID 4:[(6, 0.02977952), (8, 0.02275038), (9, 0.08272523), (18, 0.3232634), (19, 0.5264793)]
document ID 5:[(1, 0.9831839)]
document ID 6:[(1, 0.13254835), (5, 0.018656444), (6, 0.063223474), (8, 0.021702703), (18, 0.5291558), (19, 0.21774124)]
document ID 7:[(2, 0.25102392), (4, 0.18761417), (19, 0.5082283)]
document ID 8:[(0, 0.4817866), (4, 0.20391923), (11, 0.12106812), (17, 0.10838968), (19, 0.06340479)]
document ID 9:[(0, 0.13111), (1, 0.07129542), (4, 0.13010128), (5, 0.3589507), (10, 0.06255151), (12, 0.014940288), (14, 0.07551721), (18, 0.15228957)]


In [0]:
#documentのcategory
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [0]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(0, 0.11582458), (1, 0.19862443), (5, 0.04280683), (10, 0.015930036), (18, 0.62318116)]
['trade']
ASIAN EXPORTERS FEAR DAMAGE FROM U . S .- JAPAN RIFT Mounting trade friction between the U . S . And Japan has raised fears among many of Asia ' s exporting nations that the row could inflict far - reaching economic damage , businessmen and officials said . They told Reuter correspondents in Asian capitals a U . S . Move against Japan might boost protectionist sentiment in the U . S . And lead to curbs on American imports of their products . But some exporters said that while the conflict would hurt them in the long - run , in the short - term Tokyo ' s loss might be their gain . The U . S . Has said it will impose 300 mln dlrs of tariffs on imports of Japanese electronics goods on April 17 , in retaliation for Japan ' s alleged failure to stick to a pact not to sell semiconductors on world markets at below cost . Unofficial Japanese estimates put the impact of the tariffs at 10 billion d

## 可視化

In [0]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [0]:
#全documentを学習に用いた場合結構時間がかかる(20min~)
#gensimではK個のトピックに0~K-1のidが割り振られていたのに対し，pyLDAvisでは1~Kのidが割り振られていることに注意

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [0]:
#上で可視化したモデルをgoogle drive上にsaveできる

pyLDAvis.save_html(lda_display,'vis.html')

In [0]:
!ls

sample_data  vis.html
